In [1]:
# -*- coding: utf-8 -*-
"""
@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
from selenium.webdriver.common.action_chains import ActionChains
import os
import warnings
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

warnings.filterwarnings("ignore")


In [2]:
#driver=webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')
from selenium import webdriver 



chrome_options = Options()

chrome_options.add_argument("--disable-extensions")
# linux only
# chrome_options.add_argument("--headless")
chrome_options.headless = True 

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe',options=chrome_options)

In [3]:
RL = 'https://events.tc.umn.edu/all'
driver.get(RL)
try:
    driver.maximize_window()
except:
    pass

In [4]:
try:
    time.sleep(5)
    driver.find_element_by_xpath('/html/body/div/div[4]/div/div/div/div[1]/div[2]/div[2]/p/a[2]').click()
except:
    try:
        time.sleep(5)
        driver.find_element_by_xpath('/html/body/div/div[4]/div/div/div/div[1]/div[2]/div[2]/p/a[2]').click()
    except:
        try:
            myElem = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div[4]/div/div/div/div[1]/div[2]/div[2]/p/a[2]')))
            #print "Page is ready!"
        except TimeoutException:
            pass

In [5]:
time.sleep(15)
soup = bs(driver.page_source,'lxml')
allItems= soup.find_all('div',{'class':'lw_cal_event_info'})
umtc=[]


In [6]:
time.sleep(1)
sag=0
for s in allItems:
    try:
        web=s.find('div',{'class':'lw_events_title heading heading-2'}).find('a')['href']
    except:
        web=s.find('a')['href']

    if 'http' in web:
        pass
    else:
        link='https://events.tc.umn.edu'+web
        #########################
        try:
            tim_tease=s.find('div',{'class':'lw_events_time heading heading-3'}).text
        except:
            tim_tease=''
        ###############################
        try:
            info_tease=s.find('p',{'class':'lw_events_summary'}).text.replace('\n','').strip()

        except:
            try:
                info_tease=s.find('span',{'style':'font-weight: 400;'}).text.replace('\n','').strip()
            except:
                info_tease=''
        ##############################################
        if tim_tease=='All Day':
            time_spl=''

        elif tim_tease =='':
            time_spl=''

        elif 'Weekly' in tim_tease:
            time_spl=tim_tease.split('Weekly')[0].strip()

        elif 'Daily' in tim_tease:
            time_spl=tim_tease.split('Daily')[0].strip()

        elif 'Monthly' in tim_tease:
            time_spl=tim_tease.split('Monthly')[0].strip()

        elif 'Yearly' in tim_tease:
            time_spl=tim_tease.split('Yearly')[0].strip()

        elif ('All' and 'to') in tim_tease:
            time_spl=''

        elif 'M-F' in tim_tease:
            time_spl=tim_tease.split('M-F')[0].strip()

        else:
            time_spl=tim_tease
        ##
        if time_spl=='':
            time_st=''
        else:
            zepler=time_spl.replace(' - ',' ').split(' ')
            if len(zepler)<=2:
                time_st={'type': 'general',
                          'start_time':zepler[0],
                          'end_time': '',
                          'timezone': zepler[-1],
                          'days': 'all'}
            elif len(zepler)==3:
                time_st={'type': 'general',
                          'start_time':zepler[0],
                          'end_time': zepler[1],
                          'timezone': zepler[-1],
                          'days': 'all'}

            elif len(zepler)>3:
                time_st={'type': 'general',
                          'start_time':zepler[0],
                          'end_time': zepler[1],
                          'timezone': zepler[-1],
                          'days': 'all'}
            else:
                time_st=''

        ###
        if time_st=='':
            time_=''
        else:
            time_=json.dumps([time_st])
        ##############################
        ######################
        ####################

        driver.get(link)
        soupn = bs(driver.page_source,'lxml')
        try:
            myElem = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, 'lw_cal_events')))
            #print "Page is ready!"
        except TimeoutException:
            driver.get(link)
            time.sleep(17)
            soupn = bs(driver.page_source,'lxml')
        items= soupn.find('div',{'id':'lw_cal_events'})
        flm=True
    #     if "lw_cal_loading" in items:
        while flm:
            time.sleep(5)
            soupn = bs(driver.page_source,'lxml')
            items= soupn.find('div',{'id':'lw_cal_events'})
            #print('aseko')
            if "lw_cal_loading" not in items:
                #print('otitan')
                flm=False
        lftitems= soupn.find('div',{'id':'lw_cal_event_leftcol'})
        try:
            title=items.find('h2',{'class':'lw_events_title heading heading-2'}).text
        except:
            try:
                time.sleep(5)
                soupn = bs(driver.page_source,'lxml')
                items= soupn.find('div',{'id':'lw_cal_events'})
                title=items.find('h2',{'class':'lw_events_title heading heading-2'}).text
            except:
                title=''
        ##############################
        try:
            even=lftitems.find('a',{'class':'lw_join_online umn-button-regular'})['href']
        except:
            even=''
        if even=='':
            event_web=link
        else:
            event_web=even
        ###################################
        #double check
        try:
            if lftitems.find('h3',{'class':'heading heading-3'}).text =='Contact Info:':
                sinzu=lftitems.find('p').text
                if '@' in sinzu:
                    sin=sinzu
                else:
                    sin=''
            else:
                sin=''
            if sin !='':    
                money=sin.split('\n')
                for mas in money:
                    if '@' in mas:
                        mail_=mas
            else:
                mail_=''
        except:
            mail_=''
        ###############################    
        #on_off
        try:
            onner=items.find('div',{'class':'location_marker_container'}).text.replace('\n','')
        except:
            onner=''
        if ('Online'or 'online') in onner:
            on_off='1'
        else:
            on_off='0'
        ###############################
        try:
            event_info=' '.join(soupn.find('div',{'class':'intro'}).text.split())
        except:
            if info_tease != '':
                event_info=info_tease
            else:
                event_info=''

        ###################################
        zazu=items.find_all('span',{'class':'event_info_label'})
        zeh=len(zazu)
        for n in range(1,zeh+1):
            try:
                if driver.find_element_by_xpath(f'/html/body/div/div[4]/div/div/div/div[1]/div[2]/div[2]/div/div[2]/div[3]/div[{n}]/span').text=='Time:':
                    tim_dur=driver.find_element_by_xpath(f'/html/body/div/div[4]/div/div/div/div[1]/div[2]/div[2]/div/div[2]/div[3]/div[{n}]').text.split(': ')[1]
            except NoSuchElementException:
                tim_dur=''

            try:
                if driver.find_element_by_xpath(f'/html/body/div/div[4]/div/div/div/div[1]/div[2]/div[2]/div/div[2]/div[3]/div[{n}]/span').text=='Date:':
                    dat_dur=driver.find_element_by_xpath(f'/html/body/div/div[4]/div/div/div/div[1]/div[2]/div[2]/div/div[2]/div[3]/div[{n}]').text.split(': ')[1]
            except NoSuchElementException:
                dat_dur=''

            try:
                if driver.find_element_by_xpath(f'/html/body/div/div[4]/div/div/div/div[1]/div[2]/div[2]/div/div[2]/div[3]/div[{n}]/span').text=='Location:':
                    ven=driver.find_element_by_xpath(f'/html/body/div/div[4]/div/div/div/div[1]/div[2]/div[2]/div/div[2]/div[3]/div[{n}]').text.split(': ')[1]
            except NoSuchElementException:
                ven=''
            #price    
            try:
                if driver.find_element_by_xpath(f'/html/body/div/div[4]/div/div/div/div[1]/div[2]/div[2]/div/div[2]/div[3]/div[{n}]/span').text=='Price:':
                    price=driver.find_element_by_xpath(f'/html/body/div/div[4]/div/div/div/div[1]/div[2]/div[2]/div/div[2]/div[3]/div[{n}]').text.split(': ')[1]
            except:#NoSuchElementException:
                price=''
        ####################################
        if on_off=='1':
            venue=''
            city=''
            country=''
        elif on_off=='0':
            venue=ven
            city=ven
            country="USA"
        ####################################
        if dat_dur != '':
            yr=str(date.today().year)
            if '-' in dat_dur:
                sup=dat_dur.split('-')
                spl=[q.strip()+' '+yr for q in sup]
                spl_dt_obj = [datetime.strptime(v, '%b %d %Y') for v in spl]
                date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
            else:
                for f in ['Monday','Tuesday','Wednesday','Thursday','Fiday','Saturday','Sunday']:
                    if f in dat_dur:
                        dat_dura=dat_dur.replace(f,'').replace(',','').strip()
                sup=[dat_dura, dat_dura]
                spl=[q.strip()+' '+yr for q in sup]
                spl_dt_obj = [datetime.strptime(v, '%b %d %Y') for v in spl]
                date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]

        else:
            date_=''

        if date_=='':
            start_date=end_date=''
        else:
            start_date=date_[0]
            end_date=date_[1]
        ############################################
        ticket_list=''
        org_name='University of Minnesota Twin Cities'
        org_web='https://twin-cities.umn.edu/'
        org_pro=''
        logo=''    
        sponsor=''
        agendalist=''
        type_=''
        category=''
        Speakerlist=''  

            ######################################
        def get_google_map_url(location):
            try:
                google_url_for_location="https://www.google.com/search?q="+location+"&oq="+location+"&num=1"
                time.sleep(randint(0,3))
                driver.get(google_url_for_location)
                time.sleep(4)
                try:
                    google_map_url=driver.find_element_by_id('lu_map').click()
                except:
                    try:
                        google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                    except:
                        google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
                time.sleep(1)
                google_map_url=driver.current_url
    #                 print(google_map_url)
                return(google_map_url)
            except Exception as e:
                print(location, "; didn't go through")
                print(e)
                return("")
        if 'Online' in venue or 'Webinar' in venue or 'On-line' in venue or 'Zoom' in venue or 'NHH' in venue or 'Virtua' in venue or 'online' in venue or 'webinar' in venue or 'Northro' in venue:
            googlePlaceUrl=''
            city=''
            country=''
            venue=''
            on_off='1'
        if venue!='':
            googlePlaceUrl=get_google_map_url(venue)
        else:
            googlePlaceUrl=''
    
        ############
        umtc.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                     org_pro,org_name,org_web,logo,sponsor,agendalist,
                     type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                     Speakerlist,on_off])

    sag+=1
    for d in range(0,50*100,100):
        if sag==d:
            print(d,' cards done')
print('Done')

Architecture & Landscape Architecture Library, Rapson Hall, 89 Church Street Se Minneapolis, Minnesota 55455 ; didn't go through
Message: no such element: Unable to locate element: {"method":"css selector","selector":".Lx2b0d"}
  (Session info: headless chrome=100.0.4896.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x00CC9943+2595139]
	Ordinal0 [0x00C5C9F1+2148849]
	Ordinal0 [0x00B54528+1066280]
	Ordinal0 [0x00B80FD4+1249236]
	Ordinal0 [0x00B811CB+1249739]
	Ordinal0 [0x00BAD812+1431570]
	Ordinal0 [0x00B9BA34+1358388]
	Ordinal0 [0x00BABAF2+1424114]
	Ordinal0 [0x00B9B806+1357830]
	Ordinal0 [0x00B76086+1204358]
	Ordinal0 [0x00B76F96+1208214]
	GetHandleVerifier [0x00E6B232+1658114]
	GetHandleVerifier [0x00F2312C+2411516]
	GetHandleVerifier [0x00D5F261+560433]
	GetHandleVerifier [0x00D5E366+556598]
	Ordinal0 [0x00C6286B+2173035]
	Ordinal0 [0x00C675F8+2192888]
	Ordinal0 [0x00C676E5+2193125]
	Ordinal0 [0x00C711FC+2232828]
	BaseThreadInitThunk [0x7727FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77

Done


In [10]:
len(umtc)

50

In [11]:
umtc

[['https://events.tc.umn.edu/event/7727-arbor-day-at-the-minnesota-landscape-arboretum',
  'Arbor Day at the Minnesota Landscape Arboretum',
  '2022-04-05',
  '2022-04-29',
  '',
  'Join us on Arbor Day, 9 a.m.-6 p.m. April 29, as we celebrate trees! Get outside to enjoy the Arboretum’s tree collections, and celebrate the holiday that honors the planting, upkeep and preservation of trees.',
  '',
  '',
  'University of Minnesota Twin Cities',
  'https://twin-cities.umn.edu/',
  '',
  '',
  '',
  '',
  '',
  '3675 Arboretum Drive, Chaska, MN 55318',
  'USA',
  '3675 Arboretum Drive, Chaska, MN 55318',
  'https://events.tc.umn.edu/event/7727-arbor-day-at-the-minnesota-landscape-arboretum',
  'https://www.google.com/maps/place/3675+Arboretum+Dr,+Chaska,+MN+55318,+USA/data=!4m2!3m1!1s0x87f604816a470727:0x3091ff4f58deab65?sa=X&ved=2ahUKEwigwpHboK33AhVY8rsIHSerC_YQ8gF6BAgXEAE',
  '',
  '',
  '0'],
 ['https://events.tc.umn.edu/event/7366-bud-break-5k-at-the-minnesota-landscape-arboretum',
  '

In [12]:
umtc_dict=umtc


umtc_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=umtc_dict)
umtc_df.to_csv("umtc.tsv", sep = '\t',index=False)
print('Dusted')

Dusted
